<a href="https://colab.research.google.com/github/mi-kaneyon/SlackBot_Token/blob/main/annotation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nanka kenchi

In [1]:
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib
!pip install torch==1.7.1
!pip install torchvision




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  6% |  1% |


yolact install

In [3]:
%cd /content
!git clone https://github.com/dbolya/yolact.git


/content
fatal: destination path 'yolact' already exists and is not an empty directory.


DCNv2 yukoka

In [4]:
%cd /content/yolact/external/DCNv2
!python setup.py build develop


/content/yolact/external/DCNv2
running build
running build_ext
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:352: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
building '_ext' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/yolact/external/DCNv2/src -I/usr/local/lib/python3.7/dist-packages/torch/include -I/usr/local/lib/python3.7/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.7/dist-packages/torch/include/TH -I/usr/local/lib/python3.7/dist-packages/torch/include/THC -I/usr/local/cuda/include -I/usr/include/python3.7m -c /content/yola

dataset

In [ ]:
%cd /content
!git clone https://github.com/chentinghao/download_google_drive.git

!mkdir -p '/content/yolact/weights'
weights = '/content/yolact/weights/yolact_plus_resnet50_54_800000.pth'

%cd download_google_drive
!python download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP '$weights'


/content
Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
/content/download_google_drive
32.0kB [00:00, 95.3MB/s]


omomi download

In [ ]:
%cd /content/download_google_drive
!python download_gdrive.py\
            1Jy3yCdbatgXa5YYIdTCRrSV0S9V5g1rn\
            /content/yolact/weights/resnet50-19c8e357.pth


/content/download_google_drive
97.8MB [00:00, 151MB/s]


In [2]:
!nvidia-smi

Wed Aug 10 04:37:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def train_loop(files, batch_size, model):
    size = len(files)
    for batch in range(size//batch_size):
        im_paths = files[batch*batch_size : (batch+1)*batch_size]
        X,y = load_batch(im_paths)
        # Compute prediction and loss
        output = model(X, y)
        del X
        del y
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
%cd /content/yolact/
!python train.py --config=yolact_resnet50_marinsan_config\
                 --batch_size=20\
                 --save_folder=weights/

/content/yolact
Scaling parameters by 2.50 to account for a batch size of 20.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Initializing weights...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
Traceback (most recent call last):
  File "train.py", line 504, in <module>
    train()
  File "train.py", line 307, in train
    losses = net(datum)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/parallel/data